In [ ]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install datasets bitsandbytes einops wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

if using free GPU from colab, ensure max_step must be less than 200

* bitsandbytes - for quantization of LLM
* PEFT - for fine- tuning LoRA parameter
* datset -loading data from hugging face
* wandb - for monitoring of fine tuning metrics
*

## DataSet-

* Loading the custom mental health dataset (heliosbrahma/mental_health_chatbot_dataset) from HuggingFace datasets.

* It contains only 1 column i.e. “text” which contains the conversation pair between the patient and doctor.

* Dataset consists of 172 rows of high-quality conversations between a patient and a healthcare provider

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

## Transformers -
* It is library from huggingface that has APIs and tools to easily download and train the state-of-the-art (`means most advanced, currently trending`) pre-trained models.

* Pipeline() - The pipeline() is the easiest and fastest way to use a pretrained model for inference.

* Inference - refers to the process of using a trained model to make predictions or generate outputs based on new, unseen data.

* link - https://huggingface.co/docs/transformers/en/quicktour



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
data = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
data

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})

In [ ]:
print(data["train"][0]['text'])

<HUMAN>: What is a panic attack?
<ASSISTANT>: Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.


* As we are fine tuning our model with QLoRA -

* First we get the peft_model by lora_config then we quantization it with 4bit.

* Following code is parameters that we used to load our model into a form of 4bit quantization and nf4 datatype.

In [ ]:
model_name = "ybelkada/falcon-7b-sharded-bf16" # sharded falcon-7b model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

# Load a quantized model -
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)

config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

...

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/921M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# `PEFT`
* offers parameter-efficient methods for finetuning large pretrained models.

* it use a reparametrization method like low-rank adaptation (LoRA) to reduce the number of trainable parameters.


* https://huggingface.co/docs/peft/en/quicktour

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
# Dropout is a regularization technique used to prevent overfitting

lora_rank = 32 # dimension of the low-rank matrices


# defined by a `PeftConfig` class that stores all the important parameters for building a PeftModel.

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM",
    target_modules=[         # Setting names of modules in falcon-7b model that we want to apply LoRA to(optional)
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

# Now we got the peftModel
peft_model = get_peft_model(model, peft_config)

# AutoTokenizer

* it is class or tool of transformers
* Nearly every NLP task begins with a tokenizer. A tokenizer converts your input into a format that can be processed by the model.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Setting pad_token same as eos_token

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Following code is to set the parameters for tuning the model - Traning Aurguments

In [ ]:
# output_dir = "./falcon-7b-sharded-bf16-finetuned-mental-health-conversational"
output_dir = "noman7/falcon-mental-health-therapy"
per_device_train_batch_size = 4 # batch size per GPU for traning
gradient_accumulation_steps = 8  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = 50   #(epochs = max_steps)     # training will happen for 320 steps
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    bf16=False,
    fp16 = True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
)

`Trainer` object that take model,  training arguments, training and test datasets as parameter and more.

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=132,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

In [ ]:
# authenticate WandB for logging metrics
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

### Fine tuned the model

In [ ]:
peft_model.config.use_cache = False
trainer.train()

Step,Training Loss
10,1.690800
20,1.328500
30,1.101900
40,0.907400
50,0.813900


TrainOutput(global_step=50, training_loss=1.168507957458496, metrics={'train_runtime': 1130.0797, 'train_samples_per_second': 1.416, 'train_steps_per_second': 0.044, 'total_flos': 8090857317396480.0, 'train_loss': 1.168507957458496, 'epoch': 9.3})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/noman7/falcon-mental-health-therapy/commit/6f3425d6554a10e0095d62c332c02a6480326eae', commit_message='End of training', commit_description='', oid='6f3425d6554a10e0095d62c332c02a6480326eae', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Loading PEFT model
PEFT_MODEL = "noman7/falcon-mental-health-therapy"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
def generate_answer(query):
  system_prompt = """Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

  user_prompt = f"""<HUMAN>: {query}
  <ASSISTANT>: """

  final_prompt = system_prompt + "\n" + user_prompt

  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output}')
  print(dashline)

In [ ]:
query = "How can I prevent anxiety and depression?"
generate_answer(query)

PEFT MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: How can I prevent anxiety and depression?
  <ASSISTANT>: 1. Maintain a healthy lifestyle: Eat nutritious food, get enough sleep, exercise regularly, avoid alcohol and drugs, and manage stress effectively. 2. Seek Professional Help: Talk to a mental health professional if you feel overwhelmed or have difficulty coping with stressful situations. They can provide personalized guidance and support. 3. Build Social Support Network: Reach out to friends, family members, or join groups that share similar interests. Having a strong social network can help buffer against feelings of loneliness and isolation. 4. Practice Mindfulness: Engaging in regular mindfulness practices such as meditation or yoga has been shown to reduce levels of anx

In [ ]:
query = "How to take care of mental health?"
generate_answer(query)

PEFT MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: How to take care of mental health?
  <ASSISTANT>: 1. Seek Help: Talk to someone you trust about your feelings and concerns. Reach out to friends, family members, or a professional who can provide support and understanding. Sharing your thoughts and emotions with others may help relieve some burden and stress.

2. Maintain Social Connections: Stay connected with loved ones and build meaningful relationships. Spend time with supportive people who understand and value your well-being. Having a strong social network provides emotional stability and sense of belonging.

3. Get Regular Exercise: Incorporate physical activity into your routine. Physical activity boosts mood and reduces anxiety and depression symptoms. Even simple activi

In [ ]:
query = "Could you please give me some tips on anxiety?"
generate_answer(query)

PEFT MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: Could you please give me some tips on anxiety?
  <ASSISTANT>: 1. Seek Professional Help: Consult a mental health professional like a psychologist or psychiatrist who can provide personalized guidance and support. They are trained to help individuals manage their anxiety symptoms.
2. Breathing Exercises: When faced with stressful situations, try deep breathing exercises. Inhale deeply through your nose and hold your breath for a count of four. Exhale slowly through your mouth, releasing all the stress and tension. Repeat as needed.
3. Distraction: Try engaging in activities that require complete focus, such as puzzles, games, or reading books. This will take your mind off anxious thoughts and worries.
4. Cognitive-Behavioral Thera

In [ ]:
# files.download(noman7)

In [ ]:
from google.colab import files

In [ ]:
import os
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'



In [ ]:
!zip -r noman7.zip noman7/

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("wandb", 'zip', "wandb")
files.download("wandb.zip")

KeyboardInterrupt: 